# Developing in Python

<img src="http://i.imgur.com/91PUPZA.png" width=20%>

## Conda environments


Conda is an environment and package manager. It was developed for Python, but
part of its strength is that it can manage dependencies in any language. This
includes C libraries and compilers, which many Python packages depend on. A
conda environment is a little bundle of interconnected binaries and source
code. They are isolated from each other, which lets you run code with different
versions of dependencies, without interfering with other installed software.

You can create new conda environments with the command: `conda create -n <name>
package=version [...]`, and then activate it with `source activate <name>`
(Mac/Linux) or `activate <name>` (Windows). Similarly, use `source deactivate`
or `deactivate` to stop using that environment.

The default environment when you install Anaconda is called the *conda root*
environment.


### Exercise: environments

Create a new conda environment called `minimal` with just Python 3.6. Activate
it and verify that other packages you've used till now are no longer available.
(For example, Jupyter.)


## Getting packages

The standard Python package manager is called pip. It installs Python packages
from the Python Package Index, or PyPI. You can install packages with `pip
install <pkg_name>`, and update them with `pip install -U <pkg_name>`.

Conda, in addition to managing environments, can also manage the packages
within those environments. It has at least two advantages over the more
official `pip`:

* It can manage non-Python dependencies.
* It can distribute pre-compiled binaries for all the major platforms,
  simplifying the life of end-users.

So, although `pip install numpy` and `conda install numpy` both work, you can
only `conda install boost` to get the Boost C++ library.

Use `conda install` to add new packages to the currently active environment and
`conda update` to upgrade existing packages.

One downside of conda is that its repositories are not as well-populated as
PyPI, but this is improving, particularly in a new community conda *channel*
(like a repository) called *conda-forge*.

Many Python developers, ourselves included, have settled on using conda until it
fails to find a package, then trying *conda-forge*, before falling back on pip.
(Pip installs packages correctly within conda environments.)


### Installing a package

One these *should* work for you (try in this order) on the command line:

In [ ]:
conda install <pkg>
conda install -c conda-forge <pkg>
pip install <pkg>


### upgrading a package

In [ ]:
conda update <pkg>
pip install --U <pkg>



### uninstalling

In [ ]:
conda remove <pkg>
pip uninstall <pkg>



Use `pip --help` or `conda --help` for more information.


## Creating and installing packages with setup.py

The standard way to make installable Python packages is to provide a `setup.py`
— which is kind of like a build Makefile for Python packages. The rules for
making a `setup.py` are certainly too long-winded for most users to remember.
We recommend one of two approaches:

- Refer to the Python Packaging Authority's [distributing
  guide](https://packaging.python.org/distributing/)
- Find a project you like, copy their `setup.py`.

Once you have a non-trivial Python package, it is very advantageous to write a
`setup.py` file for it, because it lets you install it, even *in place* into
conda environments, so that you can use it from any working directory.

Once you've created a setup.py file, you can:

In [ ]:
pip install .

to install the package to your currently active environment. (Note: you might
see many places recommend `python setup.py install`, but this form is
deprecated.)

You can also:

In [ ]:
	pip install -e .

to install the package *from its current location* into the current
environment. This means that changes to the code will propagate to the
installed version, because the installed version is just a link to the current
directory (with some metadata).

Finally, should you want to share your packages on PyPI, you can create a
*source distribution* with:

In [ ]:
python setup.py sdist

And then upload it with:

In [ ]:
twine upload sdist/*

(You might need to install `twine` with `pip install twine`.)


## Writing command-line interfaces

Python makes it very easy to write simple command-line utilities. Let's write
one that to find files matching certain criteria, usable like this:

`filefinder --min-size 100 --max-size 10000 -c .txt .  `

We can use `argparse`, which is included in the Python standard library. In
addition to providing convenient parsing code, `argparse` automatically
generates help and usage messages and issues errors when invalid arguments are
provided.

In [5]:
import argparse

### Setting up a parser

* First step for `argparse`: create parser object & tell it what arguments to
  expect.
* It can then be used to process the command line arguments on runtime
* Parser class: `ArgumentParser`. Takes several arguments to set up the
  description used in the help text for the program & other global behaviors

See https://docs.python.org/3/library/argparse.html

In [6]:
parser = argparse.ArgumentParser(description='Find files matching certain criteria')

We can now add arguments to the parser:

In [7]:
parser.add_argument('-m', '--min-size', type=int,
                    help='Minimum size, in bytes, of returned files.')
parser.add_argument('-M', '--max-size', type=int,
                    help='Maximum size, in bytes, of returned files.')

_StoreAction(option_strings=['-M', '--max-size'], dest='max_size', nargs=None, const=None, default=None, type=<class 'int'>, choices=None, help='Maximum size, in bytes, of returned files.', metavar=None)

... And so on. We can then use the parser's `parse_args()` method to parse a command line. By default, `parse_args` uses `sys.argv[1:]`, but we can pass a custom list of arguments.


### Exercise: making a Python command line application


* Finish writing the parser above, adding a `-c`/`--contains` option to find
  only filenames containing the given string, and a `path` positional argument
  specifying where to look for files.
* Create a package, `filefinder`, with a single file, `filefind.py`, containing
  the function below (`simple_files`), and another function, `find_files`, that
  uses `os.path.getsize` and `toolz.curried.filter` to find matching files in
  the `simple_files` stream.
* Add a function, `main`, taking no arguments, that creates the parser (as
  above), parses the command line, and executes the function you made.
* Add a `setup.py` file, and use the [`entry_points` keyword
  argument](http://python-packaging.readthedocs.io/en/latest/command-line-scripts.html#the-console-scripts-entry-point)
  to `setup()` to create command that corresponds to the `main` function.
* Use `pip install` to install your command line utility to your conda
  environment.

In [8]:
def simple_files(path):
    """Yield all simple files present in `path` and subdirectories."""
    for entry in os.scandir(path):
        if not entry.is_symlink():
            if entry.is_dir() and not entry.is_symlink():
                yield from simple_files(entry.path)
            else:
                yield entry.path

*Hint:* Here is the syntax for creating entry points:

In [ ]:
setup(
    ...
    entry_points = {
        'console_scripts': ['command=package.module:main'],
    }
    ...
)